# Преобразование Фурье

### Дискретное преобразование Фурье

In [1]:
import numpy as np

Рассмотрим формулы прямого и обратного дискретного преобразвоания Фурье:
$$X_k = \sum_{n=0}^{N-1} x_n \cdot e^{-2\pi ikn/N}$$ - прямое
$$x_n = \frac{1}{N}\sum_{k=0}^{N-1} X_k e^{2\pi ikn/N}$$ - обратное

Можно заметить, что дискретное преобразование по сути является линейной операцией, соответственно можно записать вектор $\vec{X}$ как результат произведения матрицы $M$ и вектора $\vec{x}$ $$\vec{X} = M \cdot \vec{x}$$
Элемент матрицы $M$: $e^{-i 2\pi kn/N}.$

Аналогично для обратного преобразования.

In [2]:
def DFT(x):
    x = np.asarray(x)
    N = x.shape[0]
    n = np.arange(N)[None,:]
    k = n.T
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

def DFTI(x):
    x = np.asarray(x)
    N = x.shape[0]
    k = np.arange(N)[None,:]
    n = k.T
    M = 1/N * np.exp(2j * np.pi * n * k / N)
    return np.dot(M, x)

Проверка

In [3]:
x = np.random.random(512)
np.allclose(DFT(x), np.fft.fft(x))

True

In [4]:
np.allclose(DFTI(DFT(x)), x)

True

### Быстрое преобразование Фурье

Можно разделить дискретное преобразование Фурье размера $N$ на части:
        $$X_k = \sum_{m=0}^{N/2-1} x_m \cdot e^{-2\pi ik2m/N} + \sum_{m=0}^{N/2-1} x_{2m+1} \cdot e^{-2\pi ik(2m+1)/N} = E_k + e^{-2\pi ik/N}O_k$$
Похожим образом получаем $X_{k+N/2}$:
$$X_{k+N/2} = E_k - e^{-2\pi ik/N}O_k$$
Результат получим, взяв сумму $X_k$ и $X_{k+N/2}$

In [5]:
def FFT(x):
    x = np.asarray(x)
    N = x.shape[0]
    if N <= 1: 
        return x
    even = FFT(x[0::2])
    odd =  FFT(x[1::2])
    EXP = [np.exp(-2j*np.pi*k/N)*odd[k] for k in range(N//2)]
    return [even[k] + EXP[k] for k in range(N//2)] + [even[k] - EXP[k] for k in range(N//2)]

def IFFT_N(x):
    x = np.asarray(x)
    N = x.shape[0]
    if N <= 1: 
        return x
    even = IFFT_N(x[0::2])
    odd =  IFFT_N(x[1::2])
    EXP = [np.exp(2j*np.pi*k/N)*odd[k] for k in range(N//2)]
    summ = [(even[k] + EXP[k]) for k in range(N//2)] + [(even[k] - EXP[k]) for k in range(N//2)]
    return summ

def IFFT(x):
    x = np.asarray(x)
    N = x.shape[0]
    summ = IFFT_N(x)
    return [summ[i] / N for i in range(N)]

Проверка

In [6]:
x = np.random.random(512)

np.allclose(FFT(x), np.fft.fft(x))

True

In [7]:
np.allclose(IFFT(FFT(x)), x)

True

### Сравнение

Посмотрим на время выполнения алгоритма. Видно, что FFT работает быстрее, однако до numpy ещё далеко


In [8]:
%timeit DFT(x)
%timeit FFT(x)

25.9 ms ± 773 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.3 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit np.fft.fft(x)

2.67 µs ± 66 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
